# Поиск вырубок. Инициализация.

Описывается инициализация данных. Основные моменты:

1. Основное хранилище данных - GRASS GIS.
2. В БД GRASS [импортируются растры](#import_landsat) Landsat 8.
3. Для каждой сцены [производится расчет](#scene_import) отражательной способности (i.landsat.toar).
4. [Импортируются растры](#import_planet) PlanetLabs (без расчета ToAR).
5. [Импортированы контура](#import_train) рубок за 2015-2016гг.
6. [Импортирован слой лесов](#import_forest2000) за 2000 г.(на основе данных https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.2.html).

(Импорт STRM описывается в [отдельном документе](006_TopoCorrection.ipynb), посвященном топографической коррекции)

## Вспомогательные функции и модули

Импортируем вспомогательные модули.

In [1]:
import os
import glob
import tempfile
import shutil
import datetime

import utilites
reload(utilites)

<module 'utilites' from 'utilites.pyc'>

Импортируем функции для выполнения рутинных действий.

Функция распаковки архивов. Параметры: data_file -- архив, extract_dir -- каталог, в который производится распаковка.

In [2]:
from utilites import unpack

Функции, чтобы не запутаться в том, как формируется название location (будут использоваться в разных местах кода):

In [3]:
from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)

print get_location_path()

/home/cruncher/GRASSDATA/FOREST


Функции для работы с файлами сцены:

In [4]:
from utilites import (
    filename_to_bandname,
    get_raster_list,
    find_meta
)

## Импорт данных

Сначала создадим объект GRASS, через который будем выполнять запросы.

In [5]:
from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

### Импорт Landsat<a id='import_landsat'></a>

Пусть сцены лежат в подкаталоге Data домашнего каталога. Создадим переменную SCENE_DIR, храняющую пути к этому каталогу:

In [ ]:
HOME_DIR = os.getenv("HOME")
SCENE_DIR = os.path.join(HOME_DIR, 'Data', 'LANDSAT', 'Winter')

print SCENE_DIR

Получим список сцен:

In [ ]:
scenes = glob.glob(os.path.join(SCENE_DIR, '*gz'))

print scenes

#### Создание БД GRASS <a id='create_location'></a>

Если область GRASS для работы уже создана, можно сразу переходить к [импорту сцен](#scene_import). Если нет -- создадим в домашнем каталоге каталог GRASSDATA и внутри него LOCATION для хранения сцен. Для этого распакуем во временный каталог первую сцену, прочитаем из нее геоданные и создадим соответствующий LOCATION.

In [ ]:
scene = scenes[0]
location = get_location_path()

try:
    temp_dir = tempfile.mkdtemp()
    print temp_dir
    if unpack(scene, temp_dir):
        print 'Unpacked'
    else:
        print "Can't unpack scene"
    
    # Возьмем первый попавшийся канал и прочитаем из него геоданные
    file_list = get_raster_list(temp_dir)
    geofile = os.path.join(temp_dir, file_list[0])
    print geofile
    
    cmd = 'grass70 -text -e -c %s %s' % (geofile, location)
    print cmd
    
    exitcode = os.system(cmd)
    if exitcode != 0:
        print "Can't create location"
finally:
    shutil.rmtree(temp_dir)
    print 'Removed'


### MAPSET

Поскольку у на с данные из разных источников, удобнее их хранить в различных MAPSET:
 * landsat
 * planet

In [ ]:
grs.grass.run_command('g.mapset', mapset='landsat', flags='c')
grs.grass.run_command('g.mapset', mapset='planet', flags='c')

#### Импорт сцен <a id='scene_import'></a>

Импортируем сцены и рассчитаем спектральную отражательную способность (ToAR Reflectance):

1. Распаковываем сцену во временный каталог.
2. Импортируем каналы сцены.
3. Устанавливаем значения null (нет данных).
4. Рассчитываем отражательную способность.
5. i.landsat.toar может генерировать значения яркостей пикселей, выходящие за допустимый диапазон [0, 1]. Это происходит из-за неточных параметров в метаданных. Пробежимся по всем созданным растрам и отсечем то, что выходит из допустимых пределов.
6. Создадим временной штамп
7. Прописываем другие метаданные (угол Солнца и пр.)
8. Удаляем импортированные растры, хранящие DN (уже не нужны, есть растры отражательной способности).
9. Удаляем временный каталог, содержащий распакованные сцены.

In [ ]:
# Определим для удобства функцию
def import_scene(grs, scene):
    try:
        # Шаг 1
        temp_dir = tempfile.mkdtemp()
        if unpack(scene, temp_dir):
            print 'Unpacked', scene
        else:
            print "Can't unpack scene", scene
            
        metafile = find_meta(temp_dir)
        # Создадим Timestamp по дате создания снимка
        
        #  number: Landsat Number
        #  creation: Creation timestamp
        #  date: Date
        #  sun_elev: Sun Elevation
        #  sensor: Sensor
        #  bands: Bands count
        #  sunaz: Sun Azimuth Angle
        #  time: Time
        creation = grs.grass.read_command(
            'i.landsat.toar', input='dummy', output='dummy',
            metfile=metafile, lsatmet='date', flags='p').split()
        
        metadata = {}
        meta_keys = ['number', 'creation', 'date', 'sun_elev', 'sensor',
                    'bands', 'sunaz', 'time']
        for key in meta_keys:
            value = grs.grass.read_command(
                'i.landsat.toar', input='dummy', output='dummy',
                metfile=metafile, lsatmet=key, flags='p'
            ).split()[0]
            metadata[key] = value

        # Создадим Timestamp по дате создания снимка
        # creation: "date=2015-12-31"
        creation = metadata['creation'].split('=')[1]
        y, m, d = [int(x) for x in creation.split('-')]
        creation = datetime.datetime(y, m, d)
        creation = creation.strftime('%-d %b %Y').lower()

        rasters = get_raster_list(temp_dir)
        for rst in rasters:
            # Шаг 2
            band_name = filename_to_bandname(rst)
            grs.grass.run_command('r.in.gdal', input=rst, output=band_name, overwrite=True, flags='e')
            # Не все растры будут удалены, поэтому timestamp ставим на все, чтобы не разбирать,
            # где он нужен, а где нет
            grs.grass.run_command('r.timestamp', map=band_name, date=creation)

            # Шаг 3
            grs.grass.run_command('r.null', map=band_name, setnull=0)


        # Шаг 4
        indx = band_name.rindex('B')    # Названия всех каналов устроены одинаково
        prefix = band_name[:indx + 1]   # поэтому для вычисления префикса можно использовать любой
        toar_pref = 'toar_' + prefix
        
        grs.grass.run_command('g.region', raster=prefix+'1') # Охват одинаковый у всех каналов
        grs.grass.run_command('i.landsat.toar', input=prefix, output=toar_pref,
               metfile=metafile, sensor='oli8', overwrite=True)
        # Коррекция + Timestamp
        for rst in grs.grass.list_strings('rast', mapset='landsat', pattern=toar_pref+'*'):
            # Шаг 5
            rst_name = rst.split('@')[0]
            if rst_name[-2:] in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']:
                # print "%s = max(min(%s, 1.0), 0.0)" % (rst_name, rst)
                grs.grass.run_command('r.mapcalc',
                              expression="%s = max(min(%s, 1.0), 0.0)" % (rst_name, rst),
                              overwrite=True)

            # Шаг 6
            grs.grass.run_command('r.timestamp', map=rst, date=creation)
            # Шаг 7
            for key in meta_keys:
                meta = "METADATA: %s" % (metadata[key], )
                grs.grass.run_command('r.support', map=rst, hist=meta)
            

        # Шаг 8
        grs.grass.run_command('g.remove', type='rast', 
                              pattern='%s*' % (prefix, ), 
                              exclude="*BQA",
                              flags='f')


    finally:
        # Шаг 9
        shutil.rmtree(temp_dir)
        print 'Removed', scene

In [ ]:
creation ='-'
for rst in grs.grass.list_strings('rast', mapset='landsat', pattern='toar_LC81120272015365LGN00_B*'):
    print rst


Переключимся в MAPSET landsat и импортируем данные:

In [ ]:
grs.grass.run_command('g.mapset', mapset='landsat')

for scene in scenes:
    # print scene
    import_scene(grs, scene)

#### Группы растров

Создадим группы растров для удобства работы (многие GRASSовские модули обработки изображений "хотят" на входе группу).

In [ ]:
print grs.grass.read_command('g.mapset', flags='p')
rasters = grs.grass.list_strings("rast")
print rasters[:10]

In [ ]:
group_names = set([name.rpartition('_')[0] for name in rasters])
group_names = list(group_names)

for gname in group_names:
    rnames = grs.grass.read_command('g.list', type='raster', 
                              pattern=gname+'*', 
                              separator=',')
    rnames = rnames.strip()
    grs.grass.run_command('g.remove', type='group', name=gname, flags='f')  # Удалим группу, если она есть
    grs.grass.run_command('i.group', group=gname, input=rnames)

In [ ]:
groups = grs.grass.read_command('g.list', type='group', mapset='landsat')
print groups

In [ ]:
example = grs.grass.read_command('i.group', group=gname, flags='lg')
print gname

### Импорт данных PlanetLabs<a id='import_planet'></a>

Пропишем пути:

In [ ]:
HOME_DIR = os.getenv("HOME")
SCENE_DIR = os.path.join(HOME_DIR, 'Data', 'PLANET')

print SCENE_DIR

Функция импорта сцены (она значительно проще, чем фукнция импорта LANDSAT, поскольку данные хранятся в виде многоканального Geotiff). При этом там присутствует альфа-канал, в котором замаскированны значения "нет-данных": все, что лежит вне диапазона 255 альфа-канала, является недостоверным и должно быть сброшено в null.

In [ ]:
# Определим для удобства функцию
def import_planet_scene(grs, scene):
    
    name = filename_to_bandname(scene)
    print name
    grs.grass.run_command('r.in.gdal', input=scene, output=name, overwrite=True, flags='e')
    # Timestamp
    y, m, d = int(name[:4]), int(name[4:6]), int(name[6:8])
    creation = datetime.datetime(y, m, d)
    creation = creation.strftime('%-d %b %Y').lower()
    
    grs.grass.run_command('g.region', raster=name + '.red')
    
    
    # Установим null по альфа-каналу:
    for suffix in ['red', 'green', 'blue']:
        expr = "%s.%s = if(%s.alpha==255, %s.%s, null())" % (name, suffix, name, name, suffix)
        grs.grass.run_command('r.mapcalc', expression=expr, overwrite=True)
        grs.grass.run_command('r.timestamp', map="%s.%s" % (name, suffix), date=creation)
    
    grs.grass.run_command('r.composite', 
                          red=name + '.red',
                          green=name + '.green',
                          blue=name + '.blue',
                          output=name + '.composite',
                          levels=256,
                          overwrite=True
    )
       

Запускаем импорт:

In [ ]:
grs.grass.run_command('g.mapset', mapset='planet')

scenes = glob.glob(os.path.join(SCENE_DIR, '*tif'))

for scene in scenes:
    import_planet_scene(grs, scene)

In [ ]:
grs.grass.run_command('g.mapset', mapset='planet')
grs.grass.run_command('g.region', res='3')
print grs.grass.list_strings('rast', mapset='planet')
print grs.grass.read_command('r.info', map='20160215_051104_0b0f_visual.red')

### Импорт рубок<a id='import_train'></a>

In [ ]:
grs.grass.run_command('g.mapset', mapset='PERMANENT')

Пусть данные по рубкам лежат в каталоге "Data/ALARM/trainings2016".

In [ ]:
train_file = os.path.join(
    HOME_DIR,'Data/ALARM/trainings2016', 'training2015-2016_winter.shp')

In [ ]:
grs.grass.run_command(
    'v.in.ogr', input=train_file,
    output='train15_16', overwrite=True
)

In [ ]:
grs.grass.run_command("v.db.addcolumn", map="train15_16", columns="jdate int, jday int, jyear int")
grs.grass.run_command('v.db.update', map='train15_16', column='jdate', query_column="date")
grs.grass.run_command('v.db.update', map='train15_16', column='jyear', query_column="substr(date, 1, 2)")
grs.grass.run_command('v.db.update', map='train15_16', column='jday', query_column="substr(date, 3)")

Добавим в БД координаты центроида рубки.

In [ ]:
grs.grass.run_command("v.db.addcolumn", map="train15_16", columns="x double, y double")
grs.grass.run_command("v.to.db", map="train15_16", 
                      type="centroid", option="coor", columns="x,y")

In [ ]:
rows = grs.grass.read_command("v.db.select", map="train15_16", flags='c')
rows = rows.split()
for row in rows[:5]:
    print row

### Импорт слоя лесов<a id='import_forest2000'></a>

Сначала импортируем данные по лесному покрову в область с системой координат Широта/Долгота, а затем перепроецируем их в отдельную область с системой кооднитат UTM.

In [9]:
grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_ll_location_name()
)

grs.grass.run_command('g.mapset', mapset='treecover', flags='c')

0

Импортируем данные:

In [10]:
HOME_DIR = os.getenv("HOME")
DATA_DIR = os.path.join(HOME_DIR, 'Data', 'UMD', 'TreeCover')
name = os.path.join(DATA_DIR, 'Hansen_GFC2015_treecover2000_50N_130E.tif')

grs.grass.run_command('r.in.gdal', input=name, output='treecover', overwrite=True)

0

Перепроецируем, для этого переключимся в область UTM и создадим отдельный набор данных для хранения treecover:

In [11]:
grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

grs.grass.run_command('g.mapset', mapset='treecover', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

treecover



Установим регион, покрывающий все интересующие нас сцены:

In [12]:
grs.grass.run_command('g.region', region='all_scenes@landsat')

0

Перепроецируем:

In [13]:
grs.grass.run_command('r.proj', location=get_ll_location_name(), mapset='treecover', 
                      input='treecover', output='treecover', overwrite=True)

0

#### Создание маски лесов

Переклассифицируем слой в маску лесов, пометив 0 (не лес) все, что покрыто лесами менее чем на 30%:

In [19]:
rules = """
0 thru 30	= 0		non-forest
* 			= 1		forest
"""

grs.grass.write_command('r.reclass', 
                        input='treecover', rules='-', output='forest.mask.30', stdin=rules, 
                        overwrite=True)

print grs.grass.read_command('r.report', map='forest.mask.30', units='c')

+-----------------------------------------------------------------------------+
|                         RASTER MAP CATEGORY REPORT                          |
|LOCATION: FOREST                                     Wed Jan 25 15:24:03 2017|
|-----------------------------------------------------------------------------|
|          north: 5374515    east: 708015                                     |
|REGION    south: 4978485    west: 350985                                     |
|          res:        30    res:      30                                     |
|-----------------------------------------------------------------------------|
|MASK: none                                                                   |
|-----------------------------------------------------------------------------|
|MAP: Reclass of treecover in treecover (forest.mask.30 in treecover)         |
|-----------------------------------------------------------------------------|
|                       Category Informa